## Análise de dados: Comércio eletrônico brasileiro

Este projeto é composto por um conjunto de dados públicos de comércio eletrônico brasileiro, disponibilizados pelo site Olist, são registros que compõem todo o processo de venda de um produto, da compra, pagamento, entrega e avaliação, além de dados de geolocalização, produtos e vendedores. Estas informações serão tratadas e analisadas, de modo a responder questões de negócio.


### Importação de bibliotecas

In [469]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc


### Criação e iniciação de uma sessão Spark

In [470]:
spark = SparkSession.builder.appName('PySpark - Olist').getOrCreate()
spark


### Criação dos datasets a partir da leitura dos arquivos *.csv


In [471]:
df_orders = spark.read.csv('dados\olist_orders_dataset.csv', sep=',', header=True, 
                           encoding='utf-8', inferSchema=True)
df_customers = spark.read.csv('dados\olist_customers_dataset.csv', sep=',', header=True, 
                              encoding='utf-8', inferSchema=True)
df_geolocation = spark.read.csv('dados\olist_geolocation_dataset.csv', sep=',', header=True, 
                                encoding='utf-8', inferSchema=True)
df_order_items = spark.read.csv('dados\olist_order_items_dataset.csv', sep=',', header=True, 
                                encoding='utf-8', inferSchema=True)
df_order_payments = spark.read.csv('dados\olist_order_payments_dataset.csv', sep=',', header=True, 
                                   encoding='utf-8', inferSchema=True)
df_order_reviews = spark.read.csv('dados\olist_order_reviews_dataset.csv', sep=',', header=True, 
                                  encoding='utf-8', inferSchema=True, 
                                  multiLine=True, ignoreLeadingWhiteSpace=True, escape='"', quote='"')
df_products = spark.read.csv('dados\olist_products_dataset.csv', sep=',', header=True, 
                             encoding='utf-8', inferSchema=True)
df_sellers = spark.read.csv('dados\olist_sellers_dataset.csv', sep=',', header=True, 
                            encoding='utf-8', inferSchema=True)

### Verificando os tipos das colunas

In [472]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [473]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [474]:
df_geolocation.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [475]:
df_order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [476]:
df_order_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [477]:
df_order_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)



In [478]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [479]:
df_sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



### Verificando a existência de registros nulos

In [480]:
def check_nulls(dataframe, name):
    '''
    Verifica e exibe campos e quantidade de registros nulos.

    :param dataframe: string
    :param name: string
    '''
    print('\n', name.upper(), '-'* (100 - len(name)))
    for coluna in dataframe.columns:
        qty = dataframe.filter(dataframe[coluna].isNull()).count()
        if qty >= 1:
            print('', coluna, qty)


In [481]:
check_nulls(df_orders, 'df_orders')
check_nulls(df_customers, 'df_customers')
check_nulls(df_geolocation, 'df_geolocation')
check_nulls(df_order_items, 'df_order_items')
check_nulls(df_order_payments, 'df_order_payments')
check_nulls(df_order_reviews, 'df_order_reviews')
check_nulls(df_products, 'df_products')
check_nulls(df_sellers, 'df_sellers')



 DF_ORDERS -------------------------------------------------------------------------------------------
 order_approved_at 160
 order_delivered_carrier_date 1783
 order_delivered_customer_date 2965

 DF_CUSTOMERS ----------------------------------------------------------------------------------------

 DF_GEOLOCATION --------------------------------------------------------------------------------------

 DF_ORDER_ITEMS --------------------------------------------------------------------------------------

 DF_ORDER_PAYMENTS -----------------------------------------------------------------------------------

 DF_ORDER_REVIEWS ------------------------------------------------------------------------------------
 review_comment_title 87658
 review_comment_message 58256

 DF_PRODUCTS -----------------------------------------------------------------------------------------
 product_category_name 610
 product_name_lenght 610
 product_description_lenght 610
 product_photos_qty 610
 product_wei


Foram identificados valores nulos em 3 dataframes, df_orders, df_order_reviews e df_products, entretando no caso de **df_orders** os dados representam operações de venda, logo possui vários estágios podendo ser uma venda concluída, cancelada, processamento ou mesmo em trânsito, ou seja, dependendo do estágio algumas colunas podem ficarem vazias (nulas), em **df_order_reviews** há campos com reviews dos compradores sobre suas compras, não é obrigatório um cliente escrever um review e em **df_products** há produtos com nome e descrição ausentes, porém constam em pedidos de clientes.

### Verificando a existência de registros duplicados

In [482]:
def check_duplicates(dataframe, fields):
    '''
    Verifica e exibe uma amostra de 5 registros duplicados.

    :param dataframe: string
    :param fileds: string/list
    '''
    duplicate = dataframe.groupBy(fields) \
        .agg(count('*').alias('qty')) \
        .where(col('qty') > 1) \
        .orderBy(desc('qty'))   
    duplicate.show(5, truncate=False)

In [483]:
check_duplicates(df_customers, 'customer_unique_id')

+--------------------------------+---+
|customer_unique_id              |qty|
+--------------------------------+---+
|8d50f5eadf50201ccdcedfb9e2ac8455|17 |
|3e43e6105506432c953e165fb2acf44c|9  |
|ca77025e7201e3b30c44b472ff346268|7  |
|1b6c7548a2a1f9037c1fd3ddfed95f33|7  |
|6469f99c1f9dfae7733b25662e7f1782|7  |
+--------------------------------+---+
only showing top 5 rows



Em **df_customers** existem dados duplicados na coluna _customer_unique_id_, porém conforme a descrição da tabela no site Kaggle está correto conforme regra estabelecida pela Olist, basicamente esse campo permite que se identifique clientes que fizeram recompras.

In [484]:
check_duplicates(df_geolocation, ['geolocation_lat', 'geolocation_lng'])

+-------------------+------------------+---+
|geolocation_lat    |geolocation_lng   |qty|
+-------------------+------------------+---+
|-27.102098999999946|-48.62961349999995|314|
|-23.495901469908656|-46.87468669635919|190|
|-23.506049208479613|-46.71737739541604|141|
|-23.490617505282753|-46.86900366603934|127|
|-23.00551425914832 |-43.37596441256672|102|
+-------------------+------------------+---+
only showing top 5 rows



In [485]:
df_geolocation = df_geolocation.dropDuplicates()
        
df_geolocation.show(5, truncate=False)

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat    |geolocation_lng    |geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|1020                       |-23.551247650297142|-46.628331542472104|sao paulo       |SP               |
|1015                       |-23.548031800450076|-46.63357615958799 |sao paulo       |SP               |
|1033                       |-23.54077242862274 |-46.63658627751788 |são paulo       |SP               |
|1122                       |-23.529995958364616|-46.64040623812689 |sao paulo       |SP               |
|1103                       |-23.539337475993985|-46.62899422217096 |são paulo       |SP               |
+---------------------------+-------------------+-------------------+----------------+-----------------+
only showing top 5 rows



Existiam linhas duplicadas em **df_geolocation**, porém nesse dataframe não há uma coluna com um identificador exclusivo para cada linha então foi criada uma tabela com os dados exclusivos baseado nas colunas _geolocation_lat_ e _geolocation_lng_ e substituído dataframe anterior.

In [491]:
check_duplicates(df_order_items, 'order_id')

+--------------------------------+---+
|order_id                        |qty|
+--------------------------------+---+
|8272b63d03f5f79c56e9e4120aec44ef|21 |
|1b15974a0141d54e36626dca3fdc731a|20 |
|ab14fdcfbe524636d65ee38360e22ce8|20 |
|428a2f660dc84138d969ccd69a0ab6d5|15 |
|9ef13efd6949e4573a18964dd1bbe7f5|15 |
+--------------------------------+---+
only showing top 5 rows



Há linhas duplicadas em **df_order_items**, este dataframe contêm os itens que compõem uma compra, logo uma ordem de venda pode conter um ou vários produtos distintos ou não, portanto não há necessidade de efetuar qualquer ajuste.

In [492]:
check_duplicates(df_order_payments, 'order_id')

+--------------------------------+---+
|order_id                        |qty|
+--------------------------------+---+
|fa65dad1b0e818e3ccc5cb0e39231352|29 |
|ccf804e764ed5650cd8759557269dc13|26 |
|285c2e15bebd4ac83635ccc563dc71f4|22 |
|895ab968e7bb0d5659d16cd74cd1650c|21 |
|ee9ca989fc93ba09a6eddc250ce01742|19 |
+--------------------------------+---+
only showing top 5 rows



O dataframe **df_order_payments** consiste na cadastro de meios de pagamentos, parcelas (quando aplicável), então uma ordem de venda pode ser com uma ou várias formas de pagamento e estas podem ser parceladas em várias vezes, nenhuma alteração executada.

In [494]:
check_duplicates(df_order_reviews, 'order_id')

+--------------------------------+---+
|order_id                        |qty|
+--------------------------------+---+
|df56136b8031ecd28e200bb18e6ddb2e|3  |
|8e17072ec97ce29f0e1f111e598b0c85|3  |
|03c939fd7fd3b38f8485a0f95798f1f6|3  |
|c88b1d1b157a9999ce368f218a407141|3  |
|f63a31c3349b87273468ff7e66852056|2  |
+--------------------------------+---+
only showing top 5 rows



Após a conclusão da entrega do pedido o cliente recebe um e-mail convidando a efetuar uma avaliação, entretanto tanto o título quanto o mensagem da review são opcionais. Apesar de existirem pedidos com mais de uma avaliação os demais campos apresentam notas reviews, data e horas distintas, são essas informações que alimentam o dataframe **df_order_reviews**. Não foram feitas alterações.

In [496]:
check_duplicates(df_orders, 'order_id')

+--------+---+
|order_id|qty|
+--------+---+
+--------+---+



Um cliente pode efetuar um ou várias compras e que podem estar em várias estágios desde a criação da ordem até a sua entrega, ainda entre esses estágios algum outro processo pode determinar a continuidade da venda ou não, resultando em por exemplo o cancelamento, sendo assim pode ocorrer de existir registros de operações de venda ainda não concluídos, estas são as informaçõe que compõem o **df_orders**.

In [497]:
check_duplicates(df_products, 'product_id')

+----------+---+
|product_id|qty|
+----------+---+
+----------+---+



O dataframe **df_products** contém o cadastro de produtos, descrição, dimensões, entre outras informações.

In [498]:
check_duplicates(df_sellers, 'seller_id')

+---------+---+
|seller_id|qty|
+---------+---+
+---------+---+



O dataframe **df_sellers** contém o cadastro de vendedores e sua localização (cep, cidade e estado).

### Criando views temporárias para uso do Spark SQL

In [486]:
df_orders.createOrReplaceTempView('orders')
df_customers.createOrReplaceTempView('customers')
df_geolocation.createOrReplaceTempView('geolocation')
df_order_items.createOrReplaceTempView('order_items')
df_order_payments.createOrReplaceTempView('order_payments')
df_order_reviews.createOrReplaceTempView('order_reviews')
df_products.createOrReplaceTempView('products')
df_sellers.createOrReplaceTempView('sellers')


In [495]:
spark.sql('''
      SELECT * FROM order_reviews
      where order_id ='df56136b8031ecd28e200bb18e6ddb2e';
''').show(truncate=False)

+--------------------------------+--------------------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id                       |order_id                        |review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------------------+--------------------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|c444278834184f72b1484dfe47de7f97|df56136b8031ecd28e200bb18e6ddb2e|5           |null                |null                  |2017-02-08 00:00:00 |2017-02-14 13:58:48    |
|72a1098d5b410ae50fbc0509d26daeb9|df56136b8031ecd28e200bb18e6ddb2e|5           |null                |null                  |2017-02-07 00:00:00 |2017-02-10 10:46:09    |
|44f3e54834d23c5570c1d010824d4d59|df56136b8031ecd28e200bb18e6ddb2e|5           |null                |null                  |2017-02-09 00:00:00 |2017-

In [487]:
spark.sql('''
      SELECT * FROM geolocation
      where geolocation_lat='-23.57870736918802';
''').show(truncate=False)

+---------------------------+------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat   |geolocation_lng    |geolocation_city|geolocation_state|
+---------------------------+------------------+-------------------+----------------+-----------------+
|4011                       |-23.57870736918802|-46.645778615462866|sao paulo       |AC               |
|4011                       |-23.57870736918802|-46.645778615462866|são paulo       |SP               |
|4011                       |-23.57870736918802|-46.645778615462866|sao paulo       |SP               |
+---------------------------+------------------+-------------------+----------------+-----------------+



In [488]:
spark.sql('''
      SELECT * FROM customers 
      WHERE customer_unique_id='8d50f5eadf50201ccdcedfb9e2ac8455';
''').show(truncate=False)

+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|customer_id                     |customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+--------------------------------+------------------------+-------------+--------------+
|1bd3585471932167ab72a84955ebefea|8d50f5eadf50201ccdcedfb9e2ac8455|4045                    |sao paulo    |SP            |
|a8fabc805e9a10a3c93ae5bff642b86b|8d50f5eadf50201ccdcedfb9e2ac8455|4045                    |sao paulo    |SP            |
|897b7f72042714efaa64ac306ba0cafc|8d50f5eadf50201ccdcedfb9e2ac8455|4045                    |sao paulo    |SP            |
|b2b13de0770e06de50080fea77c459e6|8d50f5eadf50201ccdcedfb9e2ac8455|4045                    |sao paulo    |SP            |
|42dbc1ad9d560637c9c4c1533746f86d|8d50f5eadf50201ccdcedfb9e2ac8455|4045                    |sao paulo    |SP            |
|dfb941d6f7b02f57a44c3b7

In [489]:
spark.sql('''
      SELECT * FROM order_reviews;
''').show(30,truncate=False)

+--------------------------------+--------------------------------+------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|review_id                       |order_id                        |review_score|review_comment_title   |review_comment_message                                                                                                                                                        |review_creation_date|review_answer_timestamp|
+--------------------------------+--------------------------------+------------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|7bc2406110b926393aa56f80

In [490]:
spark.sql('''
      SELECT * FROM order_reviews WHERE order_id IS NULL;
''').show(30,truncate=False)

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+

